In [6]:
import os
import pandas as pd
import tensorflow as tf

In [7]:
def normalize_series(data, min, max):
    data-=min
    data/=max
    return data

def windowed_dataset(series, batch_size, n_past=24, n_future=4, shift=1):
    ds = tf.data.Dataset.from_tensor_slices(series)
    ds = ds.window(size=n_past+n_future, shift=shift, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(n_past+n_future))
    ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-n_future], w[-n_future:, :1]))
    ds = ds.batch(batch_size).prefetch(shift)
    return ds

In [8]:
# Retrieve dataset
df = pd.read_csv(os.path.join('./dataset/2/lettuce_dataset.csv'), sep=',', encoding='ISO-8859-1')

# Delete column date, plant_id
df = df.drop(columns=['Plant_ID', 'Date'])

# Number of features in dataset
N_FEATURES = df.shape[1]

print(df.head())
print(F'n features: {N_FEATURES}')


   Temperature (°C)  Humidity (%)  TDS Value (ppm)  pH Level  Growth Days
0              33.4            53              582       6.4            1
1              33.5            53              451       6.1            2
2              33.4            59              678       6.4            3
3              33.4            68              420       6.4            4
4              33.4            74              637       6.5            5
n features: 5


In [9]:
# Normalize data
data = df.values
split_time = int(len(data)*0.8)
data = normalize_series(data, data.min(axis=0), data.max(axis=0))

x_train = data[:split_time]
print(f'Train: {len(x_train)}')
x_valid = data[split_time:]
print(f'Test: {len(x_valid)}')


Train: 2535
Test: 634


In [10]:
# DO NOT CHANGE THIS
BATCH_SIZE = 32
N_PAST = 24  # Number of past time steps based on which future observations should be predicted
N_FUTURE = 24  # Number of future time steps which are to be predicted.
SHIFT = 1  # By how many positions the window slides to create a new window of observations.

# Code to create windowed train and validation datasets.
# Complete the code in windowed_dataset.
# YOUR CODE HERE
train_set = windowed_dataset(series=x_train, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)
# YOUR CODE HERE
valid_set = windowed_dataset(series=x_valid, batch_size=BATCH_SIZE, n_past=N_PAST, n_future=N_FUTURE, shift=SHIFT)

model = tf.keras.models.Sequential([
    # tf.keras.layers.LSTM(64, 'relu', return_sequences=True, input_shape=[N_PAST, N_FEATURES]),
    tf.keras.layers.Dense(64, input_shape=(N_PAST, N_FEATURES)),
    tf.keras.layers.Dense(32),
    tf.keras.layers.Dense(N_FUTURE, 'linear'),
])


class StopWhenReachDesireMAE(tf.keras.callbacks.Callback):
    def __init__(self, monitor='mae', monitor2='val_mae', target=0.14):
        super(StopWhenReachDesireMAE, self).__init__()
        self.monitor = monitor
        self.monitor2 = monitor2
        self.target = target

    def on_epoch_end(self, epoch, logs=None):
        current = logs.get(self.monitor)
        current2 = logs.get(self.monitor2)
        if current is not None and current2 is not None:
            if current < self.target and current2 < self.target:
                print(
                    f'\nEpoch {epoch + 1}: {self.monitor} and {self.monitor2} have reached {self.target}. Stopping training.')
                self.model.stop_training = True

stop_callback = StopWhenReachDesireMAE('mae', 'val_mae', 0.05)

# Code to train and compile the model
# YOUR CODE HERE
model.compile(
    loss='mae',
    optimizer='adam',
    metrics=['mae']
)

model.fit(
    train_set,
    epochs=60,
    validation_data=valid_set,
    callbacks=[stop_callback]
)




Epoch 1/60


      1/Unknown - 1s 1s/step - loss: 0.3662 - mae: 0.3662WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0008s vs `on_train_batch_end` time: 0.0014s). Check your callbacks.
78/78 [==============================] - 2s 7ms/step - loss: 0.1281 - mae: 0.1281 - val_loss: 0.1361 - val_mae: 0.1361
Epoch 2/60
78/78 [==============================] - 0s 4ms/step - loss: 0.0652 - mae: 0.0652 - val_loss: 0.1254 - val_mae: 0.1254
Epoch 3/60
78/78 [==============================] - 0s 4ms/step - loss: 0.0642 - mae: 0.0642 - val_loss: 0.1342 - val_mae: 0.1342
Epoch 4/60
78/78 [==============================] - 0s 4ms/step - loss: 0.0638 - mae: 0.0638 - val_loss: 0.1376 - val_mae: 0.1376
Epoch 5/60
78/78 [==============================] - 0s 4ms/step - loss: 0.0639 - mae: 0.0639 - val_loss: 0.1550 - val_mae: 0.1550
Epoch 6/60
78/78 [==============================] - 0s 4ms/step - loss: 0.0634 - mae: 0.0634 - val_loss: 0.1604 - v

In [15]:
# # Evaluate the model
# loss, mae = model.evaluate(X_test, y_test)
# print(f'Mean Absolute Error on test data: {mae}')
# 
# # Make predictions
# y_pred = model.predict(X_test)
# 
# # Optional: Convert predictions and actual values to a more readable format
# results = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred.flatten()})
# print(results.head())